In [2]:
# All Hail The Vua Spirit
# Import Modules
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [10]:
# Create The Model
class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [12]:
# Device Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [18]:
# Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.001
batch_size = 64
num_epochs = 5

In [8]:
# Loading The Dataset
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

9913344it [00:03, 3110612.92it/s]                             


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



29696it [00:00, 5335363.10it/s]          


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



1649664it [00:00, 3683468.14it/s]                             


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



5120it [00:00, 23495444.73it/s]         

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [14]:
# Initialize Model. Loss Function and Optimizer
model = NN(input_size=input_size, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
# Train
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):

        # Bring data into Device
        data = data.to(device=device)
        targets = targets.to(device=device)

        # Reshape 28 * 28 * 1 = 784
        data = data.reshape(data.shape[0], -1)

        # Loss Calculation
        scores = model(data)
        loss = criterion(scores, targets)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        
        # Gradient Decent / Adam Step
        optimizer.step()

In [20]:
# Accuracy Metrics
def check_accuracy(loader, model):
    if loader.dataset.train:
        print("Checking Accuracy on Training Data")
    else:
        print("Checking Accuracy on Test Data")
    
    num_correct = 0
    num_samples = 0

    # Set Model into Evaluation Mode
    model.eval() 

    # No Gradient Calculation Required During Evaluation
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            x = x.reshape(x.shape[0], -1)

            scores = model(x)
            # Output is 64x10, Gets the Prediction index form Output[1].max()
            # ie 64x[0,0,0,0,0,0,0,0,0.7,0] = Prediction is 8 
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
        
        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')
    
    # Return Model Into Training Mode
    model.train()

In [21]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking Accuracy on Training Data
Got 58573 / 60000 with accuracy 97.62
Checking Accuracy on Test Data
Got 9658 / 10000 with accuracy 96.58
